In [1]:
from mtv import model
from bson import ObjectId
from mongoengine import connect


connect(db='mtv', host='localhost', port=27017, username=None, password=None)

predictions = list()
doc = model.Prediction.find_one(datarun=ObjectId('5da7d08776e3e19307d0dbf6'))
data = doc.data
print('data length', len(data))
print(data[0][1])


def get_X(start_index, stop_index, window_size=100, step_size=1):
    X = list()
    while start_index <= stop_index:
        x = []
        for i in range(window_size):
            x.append(data[start_index - i][1])  # y_raw
        x.reverse()
        X.append(x)
        start_index += step_size
    return X

len1 = 50
len2 = 50
X1 = get_X(850, 850 + len1-1)
X2 = X1.copy()
# X2 = get_X(5600, 5600 + len2-1)

X = X1 + X2
# X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1]] + [[1,1,1]] * 30)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import similaritymeasures as sm

def compute_distance(x1, x2):
    x1_ = np.zeros((len(x1), 2))
    x1_[:, 0] = np.arange(len(x1))
    x1_[:, 1] = x1
    x2_ = np.zeros((len(x2), 2))
    x2_[:, 0] = np.arange(len(x2))
    x2_[:, 1] = x2
    
    dist, _ = sm.dtw(x1_, x2_)
    return dist
    
X = np.array(X)
X_embedded = TSNE(n_components=2, learning_rate=25, n_iter=5000, metric=compute_distance).fit_transform(X)
print(X_embedded.shape)

c = ['#1f77b4'] * len1 + ['#ff7f0e'] * len2

# # plt.scatter(X_embedded[:,0], X_embedded[:,1], s=area, c=colors, alpha=0.5)
plt.scatter(X_embedded[:,0], X_embedded[:,1], c=c, alpha=0.5)
plt.show()



AttributeError: 'NoneType' object has no attribute 'data'

In [12]:
from sklearn.manifold import TSNE
from mtv.resources.computing.utils.signal_distance import event_overlap_distance
from mtv.db import schema
import numpy as np
from mongoengine import connect
import matplotlib.pyplot as plt


if __name__ == '__main__':

    db = connect('mtv', host='localhost', port=27017)

    signalrun_docs = schema.Signalrun.find()
    event_list = list()
    count = 0
    for signalrun_doc in signalrun_docs:
        event_docs = schema.Event.find(signalrun=signalrun_doc)
        events = [(doc.start_time, doc.stop_time) for doc in event_docs]
        event_list.append({
            'eid': count,
            'events': events,
            'start': signalrun_doc.signal.start_time,
            'end': signalrun_doc.signal.stop_time,
            'signalrun_id': str(signalrun_doc.id)
        })
        count += 1

    dist_matrix = np.empty((count, count))

    for i in range(count):
        for j in range(i + 1):
            if i == j:
                dist_matrix[i, j] = 0
            else:
                dist_matrix[i, j] = event_overlap_distance(
                    event_list[i]['events'],
                    event_list[j]['events'],
                    min(event_list[i]['start'], event_list[j]['start']),
                    max(event_list[i]['end'], event_list[j]['end'])
                )
                dist_matrix[j, i] = dist_matrix[i, j]

    def func_dist(a, b):
        return dist_matrix[int(a[0]), int(b[0])]

    X = np.arange(count)
    X = X.reshape(-1, 1).astype(int)

    X_embedded = TSNE(n_components=1, metric=func_dist).fit_transform(X)
    X_embedded = X_embedded.flatten()
    
    X_sorted = [{'i': i, 'v': v} for i, v in enumerate(X_embedded)]
    X_sorted = sorted(X_sorted, key=lambda x: x['v'])
    
    for i, d in enumerate(X_sorted):
        event_list[d['i']]['order'] = i
    
    for d in event_list:
        print(d)
    
    


{'eid': 0, 'events': [(1243274400, 1245952800), (1283623200, 1286280000), (1358834400, 1361512800), (1391817600, 1394236800), (1398967200, 1401861600), (1416247200, 1417910400), (1420394400, 1422813600)], 'start': 1222819200, 'end': 1458756000, 'signalrun_id': '5f46ebfe4e33925bbf954c74', 'order': 5}
{'eid': 1, 'events': [(1384171200, 1387173600)], 'start': 1222819200, 'end': 1416765600, 'signalrun_id': '5f46ecee4e33925bbf954c91', 'order': 4}
{'eid': 2, 'events': [(1279216800, 1282586400), (1284595200, 1286064000), (1392271200, 1394841600), (1405663200, 1407974400), (1408320000, 1410976800)], 'start': 1222819200, 'end': 1469296800, 'signalrun_id': '5f46edc74e33925bbf954ca7', 'order': 1}
{'eid': 3, 'events': [(1271419200, 1272952800), (1273860000, 1276300800), (1375142400, 1377691200), (1392184800, 1421236800), (1421409600, 1425556800), (1391817600, 1391817900), (1391817600, 1391817900), (1391817600, 1391817900), (1363521600, 1366113600), (1356696000, 1359266400), (1384646400, 1387216800